In [1]:
import requests
import json
import time
from tqdm import tqdm
import pandas as pd
import random

In [2]:
url = 'https://api.hh.ru/areas/113' # Россия
res = requests.get(url)
areas = res.json()

In [3]:
areas

{'id': '113',
 'parent_id': None,
 'name': 'Россия',
 'areas': [{'id': '1620',
   'parent_id': '113',
   'name': 'Республика Марий Эл',
   'areas': [{'id': '4228',
     'parent_id': '1620',
     'name': 'Виловатово',
     'areas': []},
    {'id': '1621', 'parent_id': '1620', 'name': 'Волжск', 'areas': []},
    {'id': '1622', 'parent_id': '1620', 'name': 'Звенигово', 'areas': []},
    {'id': '4229', 'parent_id': '1620', 'name': 'Знаменский', 'areas': []},
    {'id': '61', 'parent_id': '1620', 'name': 'Йошкар-Ола', 'areas': []},
    {'id': '4230', 'parent_id': '1620', 'name': 'Кельмаксола', 'areas': []},
    {'id': '4231', 'parent_id': '1620', 'name': 'Килемары', 'areas': []},
    {'id': '1623', 'parent_id': '1620', 'name': 'Козьмодемьянск', 'areas': []},
    {'id': '4232',
     'parent_id': '1620',
     'name': 'Красногорский (Республика Марий Эл)',
     'areas': []},
    {'id': '4233',
     'parent_id': '1620',
     'name': 'Краснооктябрьский',
     'areas': []},
    {'id': '4234', 'pa

In [4]:
parent_ids = [el.get('id') for el in areas.get('areas')]

In [5]:
df=pd.DataFrame()
url = 'https://api.hh.ru/areas/{}'
area = []
for i in (parent_ids):
    df=df.append(pd.json_normalize(requests.get(url.format(i)).json()))
df.to_excel("area.xlsx")

In [6]:
area_ids = []
for i in df.iloc():
    for j in i['areas']:
        area_ids.append(j['id'])

In [10]:
len(area_ids)

3925

In [11]:
%%time
vacancy_ids = []
# страница
page = 0
# кол-во записей на странице
num_per_page = 100
# ссылка
for i in range(0, 20):
    for j in random.choices(area_ids, k = 1000):
        url = f'https://api.hh.ru/vacancies?page={i}&per_page={num_per_page}&area={j}'
        res = requests.get(url)
        vacancies = res.json()
        num_pages = vacancies.get('pages')
        vacancy_id = [el.get('id') for el in vacancies.get('items')]
        vacancy_ids.extend(vacancy_id)
        time.sleep(0.5)

TypeError: 'NoneType' object is not iterable

In [16]:
len(vacancy_ids)

51566

In [17]:
vacancy_ids_df=pd.DataFrame(vacancy_ids)

In [13]:
vacancy_ids_df.to_excel("vacancy_ids.xlsx")

In [19]:
vacancies_df=pd.DataFrame()
url = 'https://api.hh.ru/vacancies/{}'
for i in tqdm(random.choices(vacancy_ids, k = 10000)):
    vacancies_df=vacancies_df.append(pd.json_normalize(requests.get(url.format(i)).json()))
    time.sleep(0.0001)
vacancies_df.to_csv("vacancies_10000.csv")

100%|██████████████████████████████████████████████████████████████████████████| 10000/10000 [1:04:23<00:00,  2.59it/s]
